# Extração e Analise do IFIX


Esse projeto extrai e análisa o desempenho dos fundos de investimento que compõem o índice IFIX da B3. Este projeto realiza as seguintes etapas:

- Coleta de Dados dos Ativos: Obtém a lista de todos os fundos imobiliários que fazem parte do IFIX diretamente do site da B3.

- Download de Dados de Cotação: Utiliza a biblioteca yfinance para baixar dados históricos de cotações dos fundos listados.

- Análise de Rentabilidade: Calcula e lista os 10 fundos mais rentáveis durante o período configurado.

- Normalização dos Dados: Normaliza os dados de rentabilidade para facilitar a comparação entre os fundos.

- Visualização: Plota um gráfico mostrando a rentabilidade dos fundos, proporcionando uma visão clara e intuitiva do desempenho dos fundos.

Funcionalidades
- Listagem de Fundos do IFIX: Identifica e lista todos os fundos que compõem o índice IFIX.
- Baixa os Dados: Faz download dos dados de cotação para cada fundo usando yfinance.
- Ranking de Rentabilidade: Classifica os fundos e exibe os 10 mais rentáveis em um período especificado.
- Normalização e Plotagem: Normaliza os dados para uma comparação justa e cria gráficos de rentabilidade.


# O que é o IFIX?

O IFIX representa uma carteira teórica com os fundos imobiliários mais negociados do mercado. Esse índice foi criado pela B3, bolsa de valores brasileira, em 2012, com o objetivo de mostrar o retorno médio dos FIIs. Para isso, ele considera a variação dos preços e a distribuição de dividendos dos fundos imobiliários que compõem a carteira teórica.

A cada quatro meses, essa carteira é revista, ocasiões nas quais são retirados do índice os FIIs que não preenchem mais seus pré-requisitos, e acrescentados os que estão de acordo com os seus critérios.

# 0. Importação das bibliotecas e modulos

In [ ]:
import time
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup

import yfinance as yf

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from datetime import datetime

# 1. Pegando Dados da B3

In [ ]:
url = 'https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-fundos-de-investimentos-imobiliarios-ifix-composicao-da-carteira.htm'

Processo automatizado para baixar os dados - Dura 12 segundos

In [ ]:
driver = webdriver.Chrome()
driver.get('https://sistemaswebb3-listados.b3.com.br/indexPage/day/IFIX?language=pt-br')
time.sleep(5)

#Achando o botão
driver.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a')
time.sleep(3)

#Clicando no botão
driver.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a').click()

time.sleep(4)
driver.quit()

# 2. Tratando o arquivo baixado

In [ ]:
# Obtendo o diretório home do usuário
home_dir = Path.home()

# Construindo o caminho para a pasta Downloads
downloads_dir = home_dir / 'Downloads'

# O nome do arquivo muda com a data, então criei um nome dinamico tambem
today_date = datetime.now().strftime('%d-%m-%y')

# Construindo o nome do arquivo
filename = f'IFIXDia_{today_date}.csv'

# Construindo o caminho completo para o arquivo
file_path = downloads_dir / filename

# Verificando se o arquivo existe
if file_path.exists():
    print(f"O arquivo foi achado!")
    # Lendo o conteúdo do arquivo
    ifix = pd.read_csv(file_path, header = 1, encoding='latin-1', decimal = ',' ,sep = ';', index_col=False)
else:
    print(f"O arquivo não foi encontrado.")


**Todos os ativos do IFIX**

In [ ]:
ifix[:-2]
#Retirei as ultimas 2 linhas por que tem o cabeçalho

In [ ]:
codigos = ifix[:-2].Código
codigos

# 3. Análise dos FIIs do IFIX

Adicionando o .SA nos tickers antes de obter dados

In [ ]:
tickers = list(ifix[:-2].Código)
tickers = [i + '.SA' for i in tickers]

In [ ]:
fiis = yf.download(tickers, start = "2024-01-01")['Adj Close']
fiis

Normalizando as cotações e obtendo a última linha (mais recente), logo, a que passa a informação de rentabilidade no periodo configurado

In [ ]:
fiis_norm = fiis/fiis.iloc[0]
fiis_norm.iloc[-1]

Calculando em termos percentuais e ordenando do maior para o menor

In [ ]:
rentabilidade_fiis = (fiis_norm.iloc[-1] - 1)*100
rentabilidade_fiis.sort_values(ascending  = False)

Obtendo os 10 Fiis que mais subiram

In [ ]:
rentabilidade_fiis.sort_values(ascending  = False).head(10)

Filtrando as cotações dos 10 que mais subiram

In [ ]:
fiis[list(rentabilidade_fiis.sort_values(ascending  = False).head(10).index)].iloc[-1]

# Visualizando performance normalizada

In [ ]:
fiis_norm[list(rentabilidade_fiis.sort_values(ascending  = False).head(10).index)].plot(figsize = (10,6))